# Attendance System

In [8]:
# import cv2
# import numpy as np
# import os
# import pandas as pd
# from datetime import datetime
# from sklearn.metrics.pairwise import cosine_similarity

# # Load OpenCV face detector model
# prototxt = "deploy.prototxt"  # Model architecture
# caffemodel = "res10_300x300_ssd_iter_140000.caffemodel"  # Pre-trained weights
# net = cv2.dnn.readNetFromCaffe(prototxt, caffemodel)

# # Load known face embeddings
# path = "face_database"
# known_embeddings = []
# names = []

# for file in os.listdir(path):
#     img = cv2.imread(os.path.join(path, file))
#     img_blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), (104.0, 177.0, 123.0))
#     net.setInput(img_blob)
#     detections = net.forward()
    
#     if detections.shape[2] > 0:
#         embedding = detections[0, 0, 0, :-1]  # Extract embedding
#         known_embeddings.append(embedding)
#         names.append(os.path.splitext(file)[0])  

# # Initialize attendance CSV
# attendance_file = "attendance.csv"
# if not os.path.exists(attendance_file):
#     df = pd.DataFrame(columns=["Name", "Time"])
#     df.to_csv(attendance_file, index=False)

# # Start webcam
# cap = cv2.VideoCapture(0)

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Convert to blob and detect faces
#     blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
#     net.setInput(blob)
#     detections = net.forward()

#     for i in range(detections.shape[2]):
#         confidence = detections[0, 0, i, 2]

#         if confidence > 0.5:
#             box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
#             (x1, y1, x2, y2) = box.astype("int")

#             # Extract embedding of detected face
#             embedding = detections[0, 0, i, :-1]

#             # Compare with known faces
#             similarities = [cosine_similarity([embedding], [known])[0][0] for known in known_embeddings]
#             best_match_index = np.argmax(similarities)

#             if similarities[best_match_index] > 0.5:  # Threshold for recognition
#                 name = names[best_match_index]

#                 # Mark attendance
#                 df = pd.read_csv(attendance_file)
#                 if name not in df["Name"].values:
#                     with open(attendance_file, "a") as f:
#                         f.write(f"{name},{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

#                 # Display name on frame
#                 cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#                 cv2.putText(frame, name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

#     cv2.imshow("AI Attendance System", frame)

#     if cv2.waitKey(1) & 0xFF == ord("q"):
#         break

# cap.release()
# cv2.destroyAllWindows()


In [24]:
# import cv2
# import numpy as np
# import os
# import pandas as pd
# from datetime import datetime
# from sklearn.metrics.pairwise import cosine_similarity

# # Load OpenCV face detector model
# prototxt = "deploy.prototxt"
# caffemodel = "res10_300x300_ssd_iter_140000.caffemodel"
# net = cv2.dnn.readNetFromCaffe(prototxt, caffemodel)

# # Load face recognition model (ONNX model from OpenCV)
# recognizer = cv2.FaceRecognizerSF.create("face_recognition_sface_2021dec.onnx", "")

# # Load known face embeddings
# path = "face_database"
# known_embeddings = []
# names = []

# for file in os.listdir(path):
#     img = cv2.imread(os.path.join(path, file))
#     if img is None:
#         continue
    
#     img_blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), (104.0, 177.0, 123.0))
#     net.setInput(img_blob)
#     detections = net.forward()
    
#     if detections.shape[2] > 0:
#         x1, y1, x2, y2 = (detections[0, 0, 0, 3:7] * np.array([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])).astype("int")
#         face = img[y1:y2, x1:x2]
        
#         if face.size == 0:
#             continue
        
#         embedding = recognizer.feature(face).flatten()  # Fix: Flatten the embedding
#         known_embeddings.append(embedding)
#         names.append(os.path.splitext(file)[0])  

# # Initialize attendance CSV
# attendance_file = "attendance.csv"
# if not os.path.exists(attendance_file):
#     df = pd.DataFrame(columns=["Name", "Time"])
#     df.to_csv(attendance_file, index=False)

# # Start webcam
# cap = cv2.VideoCapture(0)

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
#     net.setInput(blob)
#     detections = net.forward()

#     for i in range(detections.shape[2]):
#         confidence = detections[0, 0, i, 2]
        
#         if confidence > 0.5:
#             x1, y1, x2, y2 = (detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])).astype("int")
#             face = frame[y1:y2, x1:x2]
            
#             if face.size == 0:
#                 continue

#             embedding = recognizer.feature(face).flatten()  # Fix: Flatten the embedding
            
#             # Compare with known faces
#             similarities = [cosine_similarity([embedding], [known])[0][0] for known in known_embeddings]
#             best_match_index = np.argmax(similarities) if similarities else -1
            
#             name = "Unknown Person"
#             if best_match_index != -1 and similarities[best_match_index] > 0.5:
#                 name = names[best_match_index]

#             # Mark attendance for known persons only
#             if name != "Unknown Person":
#                 df = pd.read_csv(attendance_file)
#                 if name not in df["Name"].values:
#                     with open(attendance_file, "a") as f:
#                         f.write(f"{name},{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

#             cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#             cv2.putText(frame, name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

#     cv2.imshow("AI Attendance System", frame)

#     if cv2.waitKey(1) & 0xFF == ord("q"):
#         break

# cap.release()
# cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np
import os
import pandas as pd
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

# Load OpenCV face detector model
prototxt = "deploy.prototxt"
caffemodel = "res10_300x300_ssd_iter_140000.caffemodel"
net = cv2.dnn.readNetFromCaffe(prototxt, caffemodel)

# Load face recognition model (ONNX model from OpenCV)
recognizer = cv2.FaceRecognizerSF.create("face_recognition_sface_2021dec.onnx", "")

# Load known face embeddings
path = "face_database"
known_embeddings = []
names = []

for file in os.listdir(path):
    img = cv2.imread(os.path.join(path, file))
    if img is None:
        continue
    
    img_blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(img_blob)
    detections = net.forward()
    
    if detections.shape[2] > 0:
        x1, y1, x2, y2 = (detections[0, 0, 0, 3:7] * np.array([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])).astype("int")
        face = img[y1:y2, x1:x2]
        
        if face.size == 0:
            continue
        
        face = cv2.resize(face, (112, 112))
        embedding = recognizer.feature(face).flatten()
        known_embeddings.append(embedding)
        names.append(os.path.splitext(file)[0])  

# Initialize attendance CSV
attendance_file = "attendance.csv"
if not os.path.exists(attendance_file):
    df = pd.DataFrame(columns=["Name", "Time"])
    df.to_csv(attendance_file, index=False)

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        
        if confidence > 0.5:
            x1, y1, x2, y2 = (detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])).astype("int")
            face = frame[y1:y2, x1:x2]
            
            if face.size == 0:
                continue

            face = cv2.resize(face, (112, 112))
            embedding = recognizer.feature(face).flatten()
            
            # Compare with known faces
            similarities = [cosine_similarity([embedding], [known])[0][0] for known in known_embeddings]
            best_match_index = np.argmax(similarities) if similarities else -1
            
            name = "Unknown Person"
            if best_match_index != -1 and similarities[best_match_index] > 0.5:
                name = names[best_match_index]

            # Update attendance log
            df = pd.read_csv(attendance_file)
            new_entry = pd.DataFrame([[name, datetime.now().strftime('%Y-%m-%d %H:%M:%S')]], columns=["Name", "Time"])
            df = pd.concat([df, new_entry], ignore_index=True)
            df.to_csv(attendance_file, index=False)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    cv2.imshow("AI Attendance System", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
